In [2]:
import os
import pandas as pd

In [123]:
data_dir = "/dataset/SV_sets/voices/Development_Data/Speaker_Recognition/sid_dev/"

### Dev Enroll List

In [282]:
dev_enroll_list = pd.read_csv("/dataset/SV_sets/voices/Development_Data/Speaker_Recognition/sid_dev_lists_and_keys/dev-enroll.lst", names=["id", "wav"], delimiter=' ')
dev_enroll_list['file'] = dev_enroll_list.wav.apply(lambda x: "/".join(x.split("/")[1:]).rstrip(".wav"))
recs =  dev_enroll_list.id.apply(lambda x: x.split('-'))
info = list(zip(*recs))[3:]
info_df = pd.DataFrame(info).T
info_df.columns = ["room", "noise", "spk_id", "ch_id", "sg_id", "mc_id", "mc_t", "mc_l", "mc_deg"]
dev_enroll_df = pd.concat([dev_enroll_list, info_df], axis=1)
dev_enroll_df.drop(columns="wav", inplace=True)
dev_enroll_df['set'] = "enroll"

In [283]:
dev_enroll_df.head()

,id,file,room,noise,spk_id,ch_id,sg_id,mc_id,mc_t,mc_l,mc_deg,set
0,Lab41-SRI-VOiCES-rm1-none-sp3446-ch144019-sg00...,sp3446/Lab41-SRI-VOiCES-rm1-none-sp3446-ch1440...,rm1,none,sp3446,ch144019,sg0006,mc03,stu,mid,dg080,enroll
1,Lab41-SRI-VOiCES-rm1-none-sp3446-ch144019-sg00...,sp3446/Lab41-SRI-VOiCES-rm1-none-sp3446-ch1440...,rm1,none,sp3446,ch144019,sg0006,mc01,stu,clo,dg080,enroll
2,Lab41-SRI-VOiCES-rm1-none-sp2093-ch143262-sg00...,sp2093/Lab41-SRI-VOiCES-rm1-none-sp2093-ch1432...,rm1,none,sp2093,ch143262,sg0015,mc01,stu,clo,dg080,enroll
3,Lab41-SRI-VOiCES-rm1-none-sp2093-ch143262-sg00...,sp2093/Lab41-SRI-VOiCES-rm1-none-sp2093-ch1432...,rm1,none,sp2093,ch143262,sg0015,mc03,stu,mid,dg080,enroll
4,Lab41-SRI-VOiCES-rm1-none-sp2269-ch088761-sg00...,sp2269/Lab41-SRI-VOiCES-rm1-none-sp2269-ch0887...,rm1,none,sp2269,ch088761,sg0004,mc03,stu,mid,dg100,enroll


### Dev Test List 

In [279]:
dev_test_list = pd.read_csv("/dataset/SV_sets/voices/Development_Data/Speaker_Recognition/sid_dev_lists_and_keys/dev-test.lst", names=["wav"])
dev_test_list['id'] = 0 # for later, ordering problem
dev_test_list['file'] = dev_test_list.wav.apply(lambda x: "/".join(x.split("/")[1:]).rstrip(".wav"))
recs =  dev_test_list.wav.apply(lambda x: x.split('-'))
info = list(zip(*recs))[3:]
info_df = pd.DataFrame(info).T
info_df.columns = ["room", "noise", "spk_id", "ch_id", "sg_id", "mc_id", "mc_t", "mc_l", "mc_deg"]
dev_test_df = pd.concat([dev_test_list, info_df], axis=1)
dev_test_df.drop(columns="wav", inplace=True)
dev_test_df["id"] = dev_test_df.file
dev_test_df['set'] = "test"

ValueError: Length mismatch: Expected axis has 20 elements, new values have 9 elements

In [ ]:
dev_test_df.head()

### Total Dev List

In [175]:
dev_df = pd.concat([dev_enroll_df, dev_test_df], sort=False)

In [285]:
dev_df.to_csv("/dataset/SV_sets/voices/Development_Data/Speaker_Recognition/sv.csv", index=False)

In [284]:
dev_df.head()

,id,file,room,noise,spk_id,ch_id,sg_id,mc_id,mc_t,mc_l,mc_deg,set
0,sp3446/Lab41-SRI-VOiCES-rm1-none-sp3446-ch1440...,sp3446/Lab41-SRI-VOiCES-rm1-none-sp3446-ch1440...,rm1,none,sp3446,ch144019,sg0006,mc03,stu,mid,dg080,enroll
1,sp3446/Lab41-SRI-VOiCES-rm1-none-sp3446-ch1440...,sp3446/Lab41-SRI-VOiCES-rm1-none-sp3446-ch1440...,rm1,none,sp3446,ch144019,sg0006,mc01,stu,clo,dg080,enroll
2,sp2093/Lab41-SRI-VOiCES-rm1-none-sp2093-ch1432...,sp2093/Lab41-SRI-VOiCES-rm1-none-sp2093-ch1432...,rm1,none,sp2093,ch143262,sg0015,mc01,stu,clo,dg080,enroll
3,sp2093/Lab41-SRI-VOiCES-rm1-none-sp2093-ch1432...,sp2093/Lab41-SRI-VOiCES-rm1-none-sp2093-ch1432...,rm1,none,sp2093,ch143262,sg0015,mc03,stu,mid,dg080,enroll
4,sp2269/Lab41-SRI-VOiCES-rm1-none-sp2269-ch0887...,sp2269/Lab41-SRI-VOiCES-rm1-none-sp2269-ch0887...,rm1,none,sp2269,ch088761,sg0004,mc03,stu,mid,dg100,enroll


In [245]:
wavs = dev_df.file.apply(lambda x: "dev_wav/"+x+".wav")
ids = dev_df.file
pd.DataFrame(ids).assign(wav=wavs).to_csv("/dataset/SV_sets/voices/wav.scp", index=False, sep=' ')
spk2utt = dev_df.sort_values("spk_id").groupby("spk_id").apply(lambda x: x.file.values)

with open("/dataset/SV_sets/voices/spk2utt", "w") as f:
    for k, v in spk2utt.iteritems():
        line = ' '.join([k]+v.tolist())
        f.write(line+'\n')

### Dev Distribution

In [166]:
# enrollemend room("rm1") and test room("rm2")
dev_df.room.value_counts()

rm2    15648
rm1      256
Name: room, dtype: int64

In [167]:
dev_df.noise.value_counts()

none    4168
tele    3912
babb    3912
musi    3912
Name: noise, dtype: int64

In [168]:
dev_df.spk_id.value_counts()

sp5400    194
sp1160    192
sp0288    192
sp0949    192
sp6241    164
sp1392    164
sp0597    162
sp2156    162
sp0472    162
sp0196    162
sp7867    162
sp0636    162
sp6415    160
sp7498    160
sp5338    160
sp0112    160
sp6895    160
sp2532    160
sp1272    160
sp4967    132
sp3645    132
sp1963    132
sp7264    130
sp5189    130
sp1851    130
sp7247    130
sp7540    130
sp1867    130
sp2758    128
sp0770    128
         ... 
sp8635     34
sp3446     34
sp1737     34
sp1052     32
sp1335     32
sp3994     32
sp7276     32
sp1182     32
sp1383     32
sp4145     32
sp8575     32
sp2074     32
sp1970     32
sp1607     32
sp0204     32
sp5157     32
sp6519     32
sp1961     32
sp2012     32
sp0307     32
sp6544     32
sp7517     32
sp4586     32
sp8605      4
sp4331      2
sp4110      2
sp2412      2
sp1212      2
sp0403      2
sp0159      2
Name: spk_id, Length: 196, dtype: int64

### VOiECE trials

In [217]:
trial_list = pd.read_csv("/dataset/SV_sets/voices/Development_Data/Speaker_Recognition/sid_dev_lists_and_keys/dev-trial-keys.lst", delimiter=" ", names=["enroll_id", "test_id", "label"])
trial_list.test_id = trial_list.test_id.apply(lambda x: x.split("/")[2].rstrip(".wav"))

In [218]:
dev_ids = dev_df.id.tolist()
id2idx = {v:i for i, v in enumerate(dev_ids)}
enr_idx = trial_list.enroll_id.apply(lambda x: id2idx[x])
test_idx = trial_list.test_id.apply(lambda x: id2idx[x])
trial_list['enroll_idx'] = enr_idx
trial_list['test_idx'] = test_idx

In [219]:
trial_list.loc[trial_list.label == 'imp', 'label'] = 0
trial_list.loc[trial_list.label == 'tgt', 'label'] = 1

In [224]:
trial_list.to_csv("/dataset/SV_sets/voices/sv_trial.csv", index=False)

In [221]:
trial_list.label.value_counts()

0    3985792
1      20096
Name: label, dtype: int64

In [223]:
print(f"target_ratio: {20096 / 3985792}")

target_ratio: 0.005041908860271685
